This is a placeholder code. You can find coding structure at https://colab.research.google.com/drive/1YDoRcSlP6irZR7vhdSxLc4XKzHTrilOo#scrollTo=47ywe8jGSKhL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/textAnalytics/train.csv')

In [ ]:
df.isnull ().sum ()

id       0
tweet    0
label    0
dtype: int64

In [ ]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from transformers import XLNetTokenizer, TFXLNetModel

class XLNetClassificationLayer(tf.keras.layers.Layer):
    def __init__(self, num_labels=2, **kwargs):
        super(XLNetClassificationLayer, self).__init__(**kwargs)
        self.num_labels = num_labels
        self.tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
        self.xlnet = TFXLNetModel.from_pretrained('xlnet-base-cased')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(num_labels, activation='softmax')

    def call(self, inputs, **kwargs):
        input_ids, attention_mask = inputs
        inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        x = self.xlnet(inputs, **kwargs)[0]
        x = self.dropout(x)
        x = tf.math.reduce_mean(x, axis=1)
        outputs = self.classifier(x)
        return outputs

    def get_config(self):
        config = super(XLNetClassificationLayer, self).get_config()
        config.update({'num_labels': self.num_labels})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
import tensorflow as tf

inputs = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
x = XLNetClassificationLayer(num_labels=2)([inputs, mask])
model = tf.keras.models.Model(inputs=[inputs, mask], outputs=x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [ ]:
model.fit(
    # {'input_ids': train_input_ids, 'attention_mask': train_attention_masks},
    [train_input_ids, train_attention_masks],
    train_labels,
    # validation_data=(
    #     # {'input_ids': val_input_ids, 'attention_mask': val_attention_masks},
    #     [val_input_ids, val_attention_masks],
    #     val_labels),
    epochs=20,
    # batch_size=32
)

Epoch 1/20


  5/308 [..............................] - ETA: 5:05 - loss: 2.0540 - accuracy: 0.5938

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/MyDrive/textAnalytics/xlnet_model.h5')
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/textAnalytics/xlnet_model.h5',
                                          custom_objects={'XLNetClassificationLayer': XLNetClassificationLayer})


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import transformers

# Load the XLNet tokenizer and model
tokenizer = transformers.XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = transformers.TFXLNetModel.from_pretrained('xlnet-base-cased')

# Load the dataset

# Convert the labels to one-hot encoded vectors
labels = pd.get_dummies(df['label']).values

# Tokenize the input texts
input_ids = []
attention_masks = []

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetMode

In [ ]:
for text in df['tweet']:
    encoded_text = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids.append(encoded_text['input_ids'])
    attention_masks.append(encoded_text['attention_mask'])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(df))
train_input_ids = input_ids[:train_size]
train_attention_masks = attention_masks[:train_size]
train_labels = labels[:train_size]

val_input_ids = input_ids[train_size:]
val_attention_masks = attention_masks[train_size:]
val_labels = labels[train_size:]

# Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
attention_masks = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)

In [ ]:
xlnet_output = model({'input_ids': input_ids, 'attention_mask': attention_masks})[0]
xlnet_output = tf.keras.layers.GlobalMaxPooling1D()(xlnet_output)
xlnet_output = tf.keras.layers.Dropout(0.2)(xlnet_output)
xlnet_output = tf.keras.layers.Dense(64, activation='relu')(xlnet_output)
xlnet_output = tf.keras.layers.Dropout(0.2)(xlnet_output)]
output = tf.keras.layers.Dense(2, activation='softmax')(xlnet_output)

model = tf.keras.models.Model(inputs=[input_ids, attention_masks], outputs=output)

# Compile the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

TypeError: ignored

In [ ]:
# Train the model
history = model.fit(
    # {'input_ids': train_input_ids, 'attention_mask': train_attention_masks},
    [train_input_ids, train_attention_masks],
    train_labels,
    # validation_data=(
    #     # {'input_ids': val_input_ids, 'attention_mask': val_attention_masks},
    #     [val_input_ids, val_attention_masks],
    #     val_labels),
    epochs=20,
    # batch_size=32
)


Epoch 1/20


 44/308 [===>..........................] - ETA: 4:18 - loss: 1.1176 - accuracy: 0.5888

KeyboardInterrupt: ignored

In [ ]:

# Evaluate the model on the test set
test_input_ids = input_ids[train_size:]
test_attention_masks = attention_masks[train_size:]
test_labels = labels[train_size:]

test_loss, test_accuracy = model.evaluate(
    # {'input_ids': test_input_ids, 'attention_mask': test_attention_masks},
    [val_input_ids, val_attention_masks],
    val_labels,
    # batch_size=32
)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

77/77 [==============================] - 28s 324ms/step - loss: 0.7064 - accuracy: 0.6654
Test Loss: 0.7063978314399719
Test Accuracy: 0.6654486656188965


In [ ]:
model.save('/content/drive/MyDrive/textAnalytics/myXlnetModel.h5')

In [ ]:
from keras.models import load_model

# load the saved model
model = load_model('/content/drive/MyDrive/textAnalytics/myXlnetModel.h5')

ValueError: ignored

In [ ]:

# Evaluate the model on the test set
test_input_ids = input_ids[train_size:]
test_attention_masks = attention_masks[train_size:]
test_labels = labels[train_size:]

test_loss, test_accuracy = loaded_model.evaluate(
    # {'input_ids': test_input_ids, 'attention_mask': test_attention_masks},
    [val_input_ids, val_attention_masks],
    val_labels,
    # batch_size=32
)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

77/77 [==============================] - 30s 330ms/step - loss: 0.9116 - accuracy: 0.5063
Test Loss: 0.9115648865699768
Test Accuracy: 0.5062931180000305
